In [ ]:
! PyInstaller --onefile -w UpdateSQL.py

In [1]:
from PyPDF4 import PdfFileReader, PdfFileWriter
import pandas as pd
import re
import os
import glob
import tabula
import pytesseract as tess
from PIL import Image
import fitz
import io
import pdfquery
from pdf2image import convert_from_path
import textract

tess.pytesseract.tesseract_cmd = r'C:\Users\ojgomes\AppData\Local\Programs\Tesseract-OCR\tesseract.exe'

In [12]:
path = 'C:\\Users\\ojgomes\\OneDrive - Stefanini\\Documents\\Sanofi\\Auto\\Auto_Dash_Sanofi\\'
os.chdir(path)

In [13]:
os.getcwd()

'C:\\Users\\ojgomes\\OneDrive - Stefanini\\Documents\\Sanofi\\Auto\\Auto_Dash_Sanofi'

In [14]:
! PyInstaller --onefile -w UpdateSQL.py

1414 INFO: PyInstaller: 4.10
1414 INFO: Python: 3.9.7 (conda)
1422 INFO: Platform: Windows-10-10.0.19043-SP0
1423 INFO: wrote C:\Users\ojgomes\OneDrive - Stefanini\Documents\Sanofi\Auto\Auto_Dash_Sanofi\UpdateSQL.spec
1427 INFO: UPX is not available.
1429 INFO: Extending PYTHONPATH with paths
['C:\\Users\\ojgomes\\OneDrive - '
 'Stefanini\\Documents\\Sanofi\\Auto\\Auto_Dash_Sanofi']
2020 INFO: checking Analysis
2172 INFO: Building because C:\Users\ojgomes\OneDrive - Stefanini\Documents\Sanofi\Auto\Auto_Dash_Sanofi\UpdateSQL.py changed
2172 INFO: Initializing module dependency graph...
2177 INFO: Caching module graph hooks...
2188 INFO: Analyzing base_library.zip ...
7381 INFO: Processing pre-find module path hook distutils from 'C:\\Users\\ojgomes\\Anaconda3\\lib\\site-packages\\PyInstaller\\hooks\\pre_find_module_path\\hook-distutils.py'.
7383 INFO: distutils: retargeting to non-venv dir 'C:\\Users\\ojgomes\\Anaconda3\\lib'
13578 INFO: Caching module dependency graph...
13973 INFO: ru

    cairo backend requires that pycairo>=1.11.0 or cairocffi is installed
365527 INFO:   Matplotlib backend "pdf": added
366088 INFO:   Matplotlib backend "pgf": added
366733 INFO:   Matplotlib backend "ps": added
367318 INFO:   Matplotlib backend "svg": added
367991 INFO:   Matplotlib backend "template": added
368851 INFO: Loading module hook 'hook-matplotlib.py' from 'C:\\Users\\ojgomes\\Anaconda3\\lib\\site-packages\\PyInstaller\\hooks'...
369295 INFO: Loading module hook 'hook-multiprocessing.util.py' from 'C:\\Users\\ojgomes\\Anaconda3\\lib\\site-packages\\PyInstaller\\hooks'...
369312 INFO: Loading module hook 'hook-numpy.py' from 'C:\\Users\\ojgomes\\Anaconda3\\lib\\site-packages\\PyInstaller\\hooks'...
369442 INFO: Import to be excluded not found: 'f2py'
369491 INFO: Loading module hook 'hook-numpy._pytesttester.py' from 'C:\\Users\\ojgomes\\Anaconda3\\lib\\site-packages\\PyInstaller\\hooks'...
369504 INFO: Loading module hook 'hook-packaging.py' from 'C:\\Users\\ojgomes\\Anaco

# O código começa aqui

In [2]:

def pdf_to_text(doc):
    images = convert_from_path(doc)


    texto = tess.image_to_string(images[0])
    #for i in range(len(images)):
   
    # Save pages as images in the pdf
    #images[i].save('page'+ str(i) +'.jpg', 'JPEG')

    #texto = tess.image_to_string(images[i])
    #break
    
    return texto


In [3]:

#função para extrair imagem do pdf e em seguida retirar texto da imagem 
def extract_text_from_img_doc(doc):
    
    file = fitz.open(doc)
    
    for page_index in range(len(file)):
        #get the page itself
        page = file[page_index]
        image_list = page.getImageList()

        #print number of image this page
        if image_list:
            print(f'[+] Found a total of {len(image_list)} imagesin page {page_index}')
        else:
            print('[!] No image found on page', page_index)
        for image_index, img in enumerate(page.getImageList(), start = 1):
            #get the XREF of the image
            xref = img[0]

            #extract image bytes
            base_image = file.extractImage(xref)
            image_bytes = base_image['image']

            #get the image extension
            image_ext = base_image['ext']

            #load it to PIL
            image = Image.open(io.BytesIO(image_bytes)).convert("RGB")
            
            #save it in local disk
            
            #image.save(open(f'image{page_index+1}_{image_index}.{image_ext}', 'wb'))
    file.close()
    texto = tess.image_to_string(image)
    return texto

# Procurando apenas por CNPJ

In [22]:
#ler pasta onde ficam os boletos
file_path = 'boletos/'

#transformando df para uma lista de todos os pdfs da pasta com a bliblioteca glob
df = glob.glob(os.path.join(file_path,'*.pdf'))

df2 = pd.read_excel('Siglas_CNPJs.xlsx')

text = ''

master_list = []

# com df como lista de pdfs itero cada pdf com for abrindo e adicionando as funções que desejo
for i in df:
    
    KeyW4 = ''
    
    #leio o pdf com pdf4, informo a quantidade de paginas e em seguida digo que quero o texto do conteudo
    pdf = PdfFileReader(i)
    obtrpage= pdf.getPage(0)
    txt = obtrpage.extractText()
    
    #encontrar CNPJ com regular expression
    KeyW4 = re.findall('\d{2}.\d{3}.\d{3}/\d{4}-\d{2}', txt)
    
    
    #caso o pdf não tenha texto e sim uma imagem não lerá PDF4, assim crie uma condição para ler imagem da msm iteração
    if len(KeyW4) == 0:
        try:
            ban_texto = pdf_to_text(i)
            KeyW4 = re.findall('\d{2}.\d{3}.\d{3}/\d{4}-\d{2}', ban_texto)
            
        except:
            print(f'erro ao ler com função pdf_to_text, boleto:  {i}')
    
    
    
    # em outra condição é tbm para imagems, mas com outra biblioteca para caso a primeira falhar
    if len(KeyW4) == 0:
        try:
            texto = extract_text_from_img_doc(i)
            KeyW4 = re.findall('\d{2}.\d{3}.\d{3}/\d{4}-\d{2}', texto)
                
        except:
            print('nenhuma imagem encontrada')
        
    #caso ainda o PDF não leia outra excelente biblioteca é tabula que ler em formato de tabelas 
    if len(KeyW4) == 0:
        try:
            #ler o pdf, converte de dataframe para lista, da lista converte para string e finalmente obtem o CNPJ com re
            pdft = tabula.read_pdf(i)
            
            pdf_list = pdft[0].iloc[:, 0].values.tolist()
            
            srt_tex = "".join(map(str, pdf_list))
            
            KeyW4 = re.findall('\d{2}.\d{3}.\d{3}/\d{4}-\d{2}', srt_tex)
            
        except:
            pass
    else:
        pass
            
    #removendo duplicidade das listas
    KeyW4 = list(dict.fromkeys(KeyW4))
    
    #removendo o item abaixo
    try:
        KeyW4.remove('42.591.651/0001-43')
    except:
        pass
    
    #KeyW4 = ','.join(KeyW4)
    
    key = [KeyW4]
    
    #este loop varre a lista key e se o item for diferente de 0 então ele aplica o Try 
    for j in key:
        if len(j)!=0:
            try:
                #tenta através de Pandas encontrar o arquivo
                search = df2[df2['CNPJ'].str.contains(j[0], case=False, na=False) | df2['CNPJ 2'].str.contains(j[0], case=False, na=False)]
                sigla = search['SIGLA'].iloc[0] + ' - ' + search['NOVO CC'].iloc[0]
                
                if len(sigla)<=12:
                    print(sigla)
                    os.rename(i, 'boletos\\' + sigla + ' - ' + os.path.basename(i))
                    break

                else:
                    print('o conteudo de sigla não é igual a 12')
            except:
                print(f'erro ao procurar keyword "{j}"')
        else:
            print('erro ')
    

    
    
    
    

IBE - B10590


In [16]:
KeyW4

[]